In [1]:
# Basic Imports
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split #serpara los datos 
from sklearn.metrics import confusion_matrix, classification_report #estos los regulatado las claisigfacion y la matriz de confusion
from sklearn.preprocessing import StandardScaler #normalizar los datos 

ModuleNotFoundError: No module named 'sklearn'

In [ ]:
def to_classlabel(z):
    return z.argmax(axis = 1) #etiqeuta en la columna

In [ ]:
#y susecion de numeros, 
def one_hot_encode(y):
    n_class = np.unique(y).shape[0] #cunatas clases tiene Y 
    y_encode = np.zeros((y.shape[0], n_class)) #inicialzia la tabla, rengloes(instancias ) columnas numero de clases
    for idx, val in enumerate(y):
        y_encode[idx, val] = 1.0
    return y_encode

In [ ]:
# Define Accuracy
def accuracy(y_true, y_pred):
    acc = np.sum(y_true == y_pred) / len(y_true)
    return acc #presicion 

In [ ]:
class Logistic_Regression():
    
    def __init__(self):
        # self.b0 = None
        self.theta = None
        
    
    def _sigmoid(self, A, theta ):
        # Linear model: yh = A * theta
        yh = np.dot(A, theta) #hipoetsis 
        # Sigmoid function 1 / (1 + e^(-yh))
        return 1/(1 + np.exp(-yh)) #funcion logistica 
    
    #fiunciond e cosot binaria 
    def _loss(self, y, h):
        '''
        a really small value 'epsilon' is added to avoid 
        overflow and divison by zero error for log
        loss = (-1/q) * sum(y * log(h) + (1-y) * log(1 - h))
        where h = 1/(1 + e^(-yh))
        '''
        #el epsislon funciona por si la hipotrsis nos da 0 (log -> logaritmo natural )
        epsilon = 1e-5
        los = (-1/len(y)) * np.sum(y * np.log(h + epsilon) + (1-y) * np.log(1-h+epsilon))
        return los

    def fit(self, A, y, learning_rate=0.01, epochs=100, batch_size=32, show_step = 10, verbose=False):
        n_obs = A.shape[0] 
        batch_loss = []
        epoch_loss = []
        
        # Initialize weights and bias
        self.theta = np.zeros(A.shape[1]) #valor de tetca inica ceros. 0 renglosn daot, columan numero de parametros 
        
        for e in range(epochs+1):                
            loss_e = 0
            
            for i in range(0, n_obs, batch_size):
                # Subset data for batches
                A_batch = A[i:i+batch_size] #matriz de disenio para el minilote 
                y_batch = y[i:i+batch_size]
                
                # Calculate loss
                y_pred = self._sigmoid(A_batch, self.theta)   # predictions
                loss = self._loss(y_batch, y_pred) #perdida 
                loss_e += loss #acumular la perdida por cada minilote
                batch_loss.append(loss) #se agrega a la lista 
                
                
                # Calculate gradients
                grad = (1/len(A_batch)) * np.dot(y_pred - y_batch, A_batch)
                
                #empieza el algritmos 
                # Update parameters
                self.theta -= learning_rate * grad
                #el optimizador devuelve los valores de tetha optmizados 


            epoch_loss.append(loss_e)
            if verbose==True and e%show_step ==0:
                print(f'Epoch: {e}, MBCE Loss: {loss_e}')
                
        return self.theta, batch_loss, epoch_loss
                
    def predict(self, A, threshold):
        y_predicted = self._sigmoid(A, self.theta)  #make prediction
        # Assign prediction to a class: 
        # if pred >= threshold then 1 else 0 and return as an array
        y_predicted_cls = [1 if i >= threshold else 0 for i in y_predicted]
        return np.array(y_predicted_cls)

In [ ]:
# Read the data
data = np.loadtxt('admisiones_dataset.txt',delimiter=',')
inputs = data[:,0:2]
idx = 2-data[:,2] #restamos el 1 para establecer el categorico, adminitivos - 1 no admitivos - 0 
targets = np.array(idx, dtype=int)     # codificacion categorica
# targets = one_hot_encode(labels)      # one hot encode to classlabel

In [ ]:
# Split the data
x_train,x_test,y_train,y_test = train_test_split(inputs,targets,test_size=0.40,random_state=1234) # test_size genreta entrenamiento y prueba 

In [ ]:
#normalizar 
# Standardize the data
scaler  = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test  = scaler.fit_transform(x_test)

In [ ]:
#matrices de disenio, 
A_train = np.c_[np.ones(len(x_train)), x_train]
A_test  = np.c_[np.ones(len(x_test)), x_test]

In [ ]:
# Build and fit best LR model
alpha = 0.01 #lr
maxEpochs = 5000
batch = 10 #minilotes
show = 500 #view

# Build model
log_model = Logistic_Regression()
# Fit Model
theta, batch_loss, epoch_loss = log_model.fit(A_train, y_train, learning_rate=alpha, 
                                epochs=maxEpochs, batch_size=batch, show_step = show, verbose=True)

Epoch: 0, MBCE Loss: 4.1315355671415395
Epoch: 500, MBCE Loss: 1.0587114401594495
Epoch: 1000, MBCE Loss: 0.8593096827207356
Epoch: 1500, MBCE Loss: 0.7721679129530061
Epoch: 2000, MBCE Loss: 0.7215413101498962
Epoch: 2500, MBCE Loss: 0.6880202653128631
Epoch: 3000, MBCE Loss: 0.6640494262916445
Epoch: 3500, MBCE Loss: 0.6460068065693545
Epoch: 4000, MBCE Loss: 0.6319181104045496
Epoch: 4500, MBCE Loss: 0.6206079359029666
Epoch: 5000, MBCE Loss: 0.6113296611515583


In [12]:
# Create predictions
train_pred = log_model.predict(A_train, 0.5)

In [ ]:
# Calculate accuracy
train_acc = accuracy(y_train, train_pred)
print(f'Accuracy on training set: {train_acc}')

Accuracy on training set: 0.9666666666666667


In [ ]:
# Calculate metrics - son de entrenamiento 
cm_train = confusion_matrix(y_train, train_pred)
train_report = classification_report(y_train, train_pred)

print("Performance on training set:\n")
print(f'Confusion Matrix:\n {cm_train}\n')
print(f'Classification Report:\n {train_report}')

Performance on training set:

Confusion Matrix:
 [[33  1]
 [ 1 25]]

Classification Report:
               precision    recall  f1-score   support

           0       0.97      0.97      0.97        34
           1       0.96      0.96      0.96        26

    accuracy                           0.97        60
   macro avg       0.97      0.97      0.97        60
weighted avg       0.97      0.97      0.97        60



In [ ]:
#los datos de prueba no usados para entrenamiento 
# Create predictions on test set
test_pred = log_model.predict(A_test, 0.5)

In [16]:
# Calculate accuracy
test_acc = accuracy(y_test, test_pred)
print(f'Accuracy on test set: {test_acc}')

Accuracy on test set: 0.8


In [17]:
# Calculate metrics
cm_test = confusion_matrix(y_test, test_pred)
test_report = classification_report(y_test, test_pred)

print("Performance on test set:\n")
print(f'Confusion Matrix:\n {cm_test}\n')
print(f'Classification Report:\n {test_report}')

Performance on test set:

Confusion Matrix:
 [[23  3]
 [ 5  9]]

Classification Report:
               precision    recall  f1-score   support

           0       0.82      0.88      0.85        26
           1       0.75      0.64      0.69        14

    accuracy                           0.80        40
   macro avg       0.79      0.76      0.77        40
weighted avg       0.80      0.80      0.80        40

